# VAE trainer

> This module handles all aspects of the VAE, including encoding, decoding, and latent space representation.

In [ ]:
#| default_exp trainers.vae_trainer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore import *
from fastcore.utils import *
from torchvision.utils import save_image
import torch
import os
from torch import nn
import pandas as pd

In [ ]:
#| export
from MAWM.trainers.trainer import Trainer
from MAWM.models.utils import save_checkpoint

class VAETrainer(Trainer):
    def __init__(self, cfg, model, train_loader, val_loader=None, 
                 criterion=None, optimizer=None, device=None,
                 dataset_train=None, dataset_test=None,
                 earlystopping=None, scheduler=None, writer= None):
        
        super().__init__(cfg, model, train_loader, val_loader, criterion, optimizer, device)
        self.dataset_train = dataset_train
        self.dataset_test = dataset_test
        self.earlystopping = earlystopping
        self.scheduler = scheduler
        self.writer = writer

        self.vae_dir = os.path.join(self.cfg.log_dir, 'vae')
        if not os.path.exists(self.vae_dir):
            os.mkdir(self.vae_dir)
            os.mkdir(os.path.join(self.vae_dir, 'samples'))

    

In [ ]:
#| export
@patch
def reload(self: VAETrainer):

    reload_file = os.path.join(self.vae_dir, 'best.pth')
    if not self.cfg.noreload and os.path.exists(reload_file):
        state = torch.load(reload_file)
        print("Reloading model at epoch {}"
            ", with test error {}".format(
                state['epoch'],
                state['precision']))
        
        self.model.load_state_dict(state['state_dict'])
        self.optimizer.load_state_dict(state['optimizer'])
        self.scheduler.load_state_dict(state['scheduler'])
        self.earlystopping.load_state_dict(state['earlystopping'])



In [ ]:
#| export
@patch
def train_epoch(self: VAETrainer, epoch):
    self.model.train()
    self.dataset_train.load_next_buffer()
    train_loss = 0
    for batch_idx, data in enumerate(self.train_loader):
        observation = data[0][0]
        # import pdb; pdb.set_trace()
        observation = observation.to(self.device)
        self.optimizer.zero_grad()
        recon_batch, mu, logvar = self.model(observation)
        loss = self.criterion(recon_batch, observation, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        self.optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(observation), len(self.train_loader.dataset),
                100. * batch_idx / len(self.train_loader),
                loss.item() / len(observation)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(self.train_loader.dataset)))

    return train_loss
       

In [ ]:
#| export
@patch
def eval_epoch(self: VAETrainer):
    self.model.eval()
    self.dataset_test.load_next_buffer()
    test_loss = 0
    with torch.no_grad():
        for data in self.val_loader:
            observation = data[0][0]
            observation = observation.to(self.device)
            recon_batch, mu, logvar = self.model(observation)
            test_loss += self.criterion(recon_batch, observation, mu, logvar).item()

    test_loss /= len(self.val_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss


In [ ]:
#| export
@patch
def fit(self: VAETrainer):
    cur_best = None
    lst_dfs = []

    for epoch in range(1, self.cfg.epochs + 1):
        train_loss = self.train_epoch(epoch)
        test_loss = self.eval_epoch()
        self.scheduler.step(test_loss)
        # self.earlystopping.step(test_loss)

        # checkpointing
        best_filename = os.path.join(self.vae_dir, 'best.pth')
        filename = os.path.join(self.vae_dir, 'checkpoint.pth')

        is_best = not cur_best or test_loss < cur_best
        if is_best:
            cur_best = test_loss

        save_checkpoint({
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'precision': test_loss,
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
            'earlystopping': self.earlystopping.state_dict()
        }, is_best, filename, best_filename)

        
        # if self.earlystopping.stop:
        #     print("End of Training because of early stopping at epoch {}".format(epoch))
        #     break

        to_log = {
            "train_loss": train_loss, 
            "test_loss": test_loss,
        }

        self.writer.write(to_log)
        df = pd.DataFrame.from_records([{"epoch": epoch ,"train_loss": train_loss, "test_loss":test_loss}], index= "epoch")
        lst_dfs.append(df)

    self.writer.finish()
    return pd.concat(lst_dfs)

In [ ]:
#| hide
import pandas as pd
df = pd.DataFrame.from_records([{"epoch": 2 ,"train_loss": 3, "test_loss":3}], index= "epoch")
df2 = pd.DataFrame.from_records([{"epoch": 3 ,"train_loss": 4, "test_loss":4}], index= "epoch")
pd.concat([df, df2])

,train_loss,test_loss
epoch,,
2,3,3
3,4,4


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()